# LVV-T1826 M2 outer control loop - closed-loop mode


The objective of this test case is to re-verify the outer loop control requirement as defined in LTS-146 after shipment from the vendor's facility to the Summit facility. This test is meant to show that the force commands given to the M2 mirror support will be converted into motor steps in order to apply them to the individual actuators.

This test case will exercise this functionality and meets the following criteria:
- Only requires the most current version of SAL
- Only requires the surrogate mirror to be installed
- Only requires the M2 to be operated in closed loop mode
- Does require telemetry data to be recorded from the EUI, EFD

Note: Using closed-loop = using the force balance system.
This test case show inadvertently that the force balance system is working correctly.
This is because when a commanded force is given such that the total net force is not zero, the force balance system will compensate by applying a pre-determined force distribution.

Link: 

 -  Test Case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1826 <br>
 -  Execution: https://jira.lsstcorp.org/secure/Tests.jspa#/testPlayer/testExecution/LVV-E2864 <br>
 -  Chronograf: https://usdf-rsp.slac.stanford.edu/chronograf/sources/1/chronograf/data-explorer?query=SELECT%20%22steps1%22%2C%20%22steps3%22%2C%20%22steps5%22%2C%20%22steps0%22%2C%20%22steps2%22%2C%20%22steps4%22%20FROM%20%22efd%22.%22autogen%22.%22lsst.sal.MTM2.tangentActuatorSteps%22%20WHERE%20time%20%3E%20%3AdashboardTime%3A%20AND%20time%20%3C%20%3AupperDashboardTime%3A

## 0. Import and Setup for the notebook


In [ ]:
test_case = "LVV-T1826" 
test_exec = "LVV-E2864" #Executed on 2023-04-19

t_start = "2023-06-14T21:48:27" # Exact time clean up initial fault
t_end = "2023-06-15T00:23:00"
#t_end = "2023-06-14T21:57:33"
t_duration = 1200
delta_t=20

In [ ]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 2
%load_ext lab_black

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
from pandas import DatetimeIndex
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient
from lsst.ts.aos.utils import DiagnosticsM2
from itertools import groupby
from matplotlib.lines import Line2D

In [ ]:
import itertools as itt
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import sys


from astropy import units as u
from astropy.time import Time, TimezoneInfo

from lsst.sitcom import vandv
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
client = EfdClient("usdf_efd")

In [ ]:
delta_t = pd.Timedelta(delta_t, "seconds")
client = vandv.efd.create_efd_client()

In [ ]:
total_axial_number = 72
total_tangent_number = 6

e_count, encoder value <br>
STEPS, total displacement in steps <br>
mm, total displacement in mm<br>
F_cmd, total force command<br>
F_cur, measured force from load cells<br>
F_HC, hardpoint compensation force<br>
F_error, force error (not retrievable from EFD)<br>

In [ ]:
# Read Encoder Position
encoder = await client.select_time_series(
    "lsst.sal.MTM2.tangentEncoderPositions",
    "*",
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"),
)

In [ ]:
# Read Axial Actuator Steps - Not actually used for this notebook.
axial_steps = await client.select_time_series(
    "lsst.sal.MTM2.axialActuatorSteps",
    "*",
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"),
)

In [ ]:
# Read Tangent Actuator Steps

tangent_steps = await client.select_time_series(
    "lsst.sal.MTM2.tangentActuatorSteps",
    "*",
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"),
)

In [ ]:
# Read Axial Forces to obtain
# 1) total force command (applied force), N), 2) measured force (N) 3) compensation force HardpointCorrection

axial_forces = await client.select_time_series(
    "lsst.sal.MTM2.axialForce",
    "*",
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"),
)

In [ ]:
# Read Tangent Forces to obtain
# 1) total force command (applied force), N), 2) measured force (N) 3) compensation force HardpointCorrection

tangent_forces = await client.select_time_series(
    "lsst.sal.MTM2.tangentForce",
    "*",
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"),
)

In [ ]:
# Read Applied forces command
applied_force_command = await client.select_time_series(
    "lsst.sal.MTM2.command_applyForces",
    "*",
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"),
)

## 1. Applied Forces and Measured Forces on each Tangent link w.r.t time 

1. Send a sequence of forces (F_cmd) 20N⁠ -20N⁠ 50N⁠ -50N⁠ 100N⁠ -100N⁠ to A1 tangent link.
2. Repeat for each tangent link (A1-A6). 

As the three active links are operated to properly distribute the load between the three active links and the three passive links, there are no motor step for A2, A4, and A6 when even the forces applies on the those tangent link 


<b>How to get/cut the window for each tangent line </b>



In [ ]:
for i in range(0, total_tangent_number):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.plot(
        tangent_forces.index,
        tangent_forces[str("".join(("applied", str(i))))],
        ".-",
        label="Applied Force",
        color="#377eb8",
    )
    ax.plot(
        tangent_forces.index,
        tangent_forces[str("".join(("measured", str(i))))]
        - np.median(tangent_forces[str("".join(("measured", str(i))))]),
        ".-",
        color="#ff7f00",
        label="Measured Force",
    )

    # Zero point of Measured forces are shifted with their median values.
    shifted_m_forces = tangent_forces[str("".join(("measured", str(i))))] - np.median(
        tangent_forces[str("".join(("measured", str(i))))]
    )
    step = tangent_steps[str("".join(("steps", str(i))))]
    applied_forces = tangent_forces[str("".join(("applied", str(i))))]

    # This is for the timestamp when forces applied on each Tangent Lines i

    applied_force_command_i = (
        np.abs(applied_force_command[str("".join(("tangent", str(i))))]) > 0
    )
    ap_force_i = applied_force_command[applied_force_command_i]
    end_range = applied_forces.index[
        (np.abs(applied_forces) <= 10)
        & (
            applied_forces.index
            > applied_forces.index[np.abs(applied_forces) >= 10][-1]
        )
        & (np.abs(shifted_m_forces) <= 10)
    ][1]

    ap_force_mean = []
    ap_force_std = []
    m_force_mean = []
    m_force_std = []
    step_mean = []
    step_std = []

    # 1. Devide the section for applied forces and measured forces for each tangential link
    for j in range(len(ap_force_i.index)):
        ax.scatter(
            ap_force_i.index[j],
            10,
            marker=r"$\downarrow$",
            s=500,
            color="m",
            linewidth=0.05,
        )
        ap_start = ap_force_i.index[j]

        if j != len(ap_force_i.index) - 1:
            ap_end = ap_force_i.index[j + 1]

        else:
            ap_end = end_range

        # Criteria : applied forces > 10N  measured forces > 5N;
        ap_forces = np.where(
            ((applied_forces.index) >= ap_start)
            & (applied_forces.index <= ap_end)
            & ((np.abs(applied_forces) > 10))
        )
        m_forces = np.where(
            ((shifted_m_forces.index) >= ap_start)
            & (shifted_m_forces.index <= ap_end)
            & ((np.abs(shifted_m_forces) > 5))
        )
       
        if len(ap_forces[0]) > 0:
            ap_force_mean.append(np.mean(applied_forces[ap_forces[0]]))
            ap_force_std.append(np.std(applied_forces[ap_forces[0]]))
           
        # For some steps, some points were missing for applied forces/measured forces, so put NaN for their average.
        else:
            ap_force_mean.append(np.nan)
            ap_force_std.append(np.nan)
          

        if len(m_forces[0]) > 0:
            m_force_mean.append(np.mean(shifted_m_forces[m_forces[0]]))
            m_force_std.append(np.std(shifted_m_forces[m_forces[0]]))
            step_forces = np.where(
                (step.index >= ap_start)
                & (
                    step.index
                    < shifted_m_forces.index[m_forces[0][-1] + 1]
                    - pd.Timedelta(1, "seconds")
                )
                & ((step > 60) | (step < -100))
            )

        else:
            m_force_mean.append(np.nan)
            m_force_std.append(np.nan)

        ax2 = ax.twinx()
        ax2.set_ylim([-1500, 1500])
        ax2.plot(step.index, step, "-", label="Steps", color="g")

        if len(step_forces[0]) > 0:
            ax2.plot(step.index[step_forces[0]], step[step_forces[0]], ".", color="k")
            step_mean.append(np.mean(step[step_forces[0]]))
            step_std.append(np.std(step[step_forces[0]]))

        else:
            step_mean.append(np.nan)
            step_std.append(np.nan)

    x_lower_limit = ap_force_i.index[0] - pd.Timedelta(20, "seconds")
    x_upper_limit = end_range + pd.Timedelta(10, "seconds")

    ax.set_xlim([x_lower_limit, x_upper_limit])

    title = (
        f"{test_case} - {test_exec}\n"
        "Applied Forces, Measured Forces, and Steps on "
        f"A{i + 1} tangent link \n"
        # f" {tangent_steps.index[0].isoformat(timespec='seconds')[:-6]} -"
        # f" {tangent_steps.index[-1].isoformat(timespec='seconds')[:-6]}")
        f" {x_lower_limit.isoformat(timespec='seconds')[:-6]} -"
        f" {x_upper_limit.isoformat(timespec='seconds')[:-6]}"
    )

    ax.set_title(f"{title}")
    ax.set_ylabel("Forces (N)", fontsize=12)
    ax2.set_ylabel("Steps", fontsize=12)
    ax.set_xlabel("Time [UTC]", fontsize=12)
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
    ax.grid(which="major", axis="both", linestyle="--")

    line1 = Line2D([0], [0], label="Applied Force", linestyle="-", color="#377eb8")
    line2 = Line2D([0], [0], label="Measured Force", linestyle="-", color="#ff7f00")
    line3 = Line2D([0], [0], label="Step", linestyle="-", color="g")
    line4 = Line2D([0], [0], label="Command to apply force", linestyle="-", color="m")

    handles = [line1, line2, line3, line4]
    ax.legend(handles=handles, loc="lower left", fontsize=12)

    fig.savefig("./Time_vs_Forces_and_Step_on_A" f"{i}.png")

    fig.tight_layout()

## 2. Linear Regression for 1) Moter Steps vs. Applied Forces 2) Moter steps vs. Measured Forces.

<b>Requirements: </b> 

1. The force commands are accepted 
2. The EUI/EFD shows the commanded force is being applied after each command
3. Plot MTM2_tangentActuatorStepsN, MTM2.tangentForce.measuredN) and plot MTM2_tangentActuatorSteps.stepsN, MTM2.tangentForce.appliedN and verify that the relation among the two variables is linear.

In [ ]:
x = np.arange(-1200, 1200, 0.5)

for i in [i for i in range(total_tangent_number) if i % 2 == 0]:
    fig, axes = plt.subplots(2, 1, figsize=(6, 8), height_ratios=[4, 1])
    ax = axes[0]

    # Zero point of Measured forces are shifted with their median values.
    shifted_m_forces = tangent_forces[str("".join(("measured", str(i))))] - np.median(
        tangent_forces[str("".join(("measured", str(i))))]
    )
    step = tangent_steps[str("".join(("steps", str(i))))]
    applied_forces = tangent_forces[str("".join(("applied", str(i))))]

    # This is for the timestamp when forces applied on each Tangent Lines i

    applied_force_command_i = (
        np.abs(applied_force_command[str("".join(("tangent", str(i))))]) > 0
    )
    ap_force_i = applied_force_command[applied_force_command_i]
    end_range = applied_forces.index[
        (np.abs(applied_forces) <= 10)
        & (
            applied_forces.index
            > applied_forces.index[np.abs(applied_forces) >= 10][-1]
        )
        & (np.abs(shifted_m_forces) <= 10)
    ][1]

    ap_force_mean = []
    ap_force_std = []
    m_force_mean = []
    m_force_std = []
    step_mean = []
    step_std = []

    # 1. Devide the section for applied forces and measured forces for each tangential link

    for j in range(len(ap_force_i.index)):
        ax.scatter(
            ap_force_i.index[j],
            10,
            marker=r"$\downarrow$",
            s=500,
            color="m",
            linewidth=0.05,
        )
        ap_start = ap_force_i.index[j]

        if j != len(ap_force_i.index) - 1:
            ap_end = ap_force_i.index[j + 1]

        else:
            ap_end = end_range

        # Criteria : applied forces > 10N  measured forces > 5N;
        ap_forces = np.where(
            ((applied_forces.index) >= ap_start)
            & (applied_forces.index <= ap_end)
            & ((np.abs(applied_forces) > 10))
        )
        m_forces = np.where(
            ((shifted_m_forces.index) >= ap_start)
            & (shifted_m_forces.index <= ap_end)
            & ((np.abs(shifted_m_forces) > 5))
        )

        if len(ap_forces[0]) > 0:
            ap_force_mean.append(np.mean(applied_forces[ap_forces[0]]))
            ap_force_std.append(np.std(applied_forces[ap_forces[0]]))

        # For some steps, some points were missing for applied forces/measured forces, so put NaN for their average.
        else:
            ap_force_mean.append(np.nan)
            ap_force_std.append(np.nan)

        if len(m_forces[0]) > 0:
            m_force_mean.append(np.mean(shifted_m_forces[m_forces[0]]))
            m_force_std.append(np.std(shifted_m_forces[m_forces[0]]))
            step_forces = np.where(
                (step.index >= ap_start)
                & (
                    step.index
                    < shifted_m_forces.index[m_forces[0][-1] + 1]
                    - pd.Timedelta(1, "seconds")
                )
                & ((step > 60) | (step < -100))
            )

        else:
            m_force_mean.append(np.nan)
            m_force_std.append(np.nan)

        if len(step_forces[0]) > 0:
            step_mean.append(np.mean(step[step_forces[0]]))
            step_std.append(np.std(step[step_forces[0]]))

        else:
            step_mean.append(np.nan)
            step_std.append(np.nan)

    ap_idx = ~np.isnan(step_mean) & ~np.isnan(ap_force_mean)
    m_idx = ~np.isnan(step_mean) & ~np.isnan(m_force_mean)

    degree = 1
    coefficients_ap = np.polyfit(
        np.array(step_mean)[ap_idx], np.array(ap_force_mean)[ap_idx], degree
    )
    coefficients_m = np.polyfit(
        np.array(step_mean)[m_idx], np.array(m_force_mean)[m_idx], degree
    )

    poly_ap = np.poly1d(coefficients_ap)
    poly_m = np.poly1d(coefficients_m)

    y_pred_ap = poly_ap(np.array(step_mean)[ap_idx])
    y_pred_m = poly_m(np.array(step_mean)[m_idx])

    rms_ap = np.array(ap_force_mean)[ap_idx] - y_pred_ap
    rms_m = np.array(m_force_mean)[m_idx] - y_pred_m

    m_ap, b_ap = np.polyfit(
        np.array(step_mean)[ap_idx], np.array(ap_force_mean)[ap_idx], 1
    )
    m_m, b_m = np.polyfit(np.array(step_mean)[m_idx], np.array(m_force_mean)[m_idx], 1)

    ax.plot(x, m_ap * x + b_ap, "--r")
    ax.plot(x, m_m * x + b_m, "--b")

    ax.errorbar(
        np.array(step_mean)[ap_idx],
        np.array(ap_force_mean)[ap_idx],
        fmt="ro",
        xerr=np.array(step_std)[ap_idx],
        yerr=np.array(ap_force_std)[ap_idx],
        label="Applied Force",
    )
    ax.errorbar(
        np.array(step_mean)[m_idx],
        np.array(m_force_mean)[m_idx],
        fmt="bo",
        xerr=np.array(step_std)[m_idx],
        yerr=np.array(m_force_std)[m_idx],
        label="Measured Force",
    )
    ax.errorbar(
        ax.get_xlim()[0] * 0.2 + ax.get_xlim()[1] * 0.8,
        ax.get_ylim()[0] * 0.8 + ax.get_ylim()[1] * 0.2,
        xerr=np.mean(np.array(step_std)[ap_idx]),
        yerr=np.mean(np.array(m_force_std)[ap_idx]),
    )

    ax.set_xlim([-1200, 1200])
    ax.text(
        ax.get_xlim()[0] * 0.55 + ax.get_xlim()[1] * 0.45,
        ax.get_ylim()[0] * 0.8 + ax.get_ylim()[1] * 0.2,
        "RMS (applied force): " + format(np.sqrt(np.mean(rms_ap**2)), ".3f"),
        bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"),
        fontsize=12,
    )

    ax.text(
        ax.get_xlim()[0] * 0.55 + ax.get_xlim()[1] * 0.45,
        ax.get_ylim()[0] * 0.83 + ax.get_ylim()[1] * 0.17,
        "RMS (measured force): " + format(np.sqrt(np.mean(rms_m**2)), ".3f"),
        bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"),
        fontsize=12,
    )

    ax.legend(fontsize=11, framealpha=1)  # ,bbox_to_anchor=(1.2, -0.2))
    ax.grid(which="major", axis="both", linestyle="--")

    title = (
        f"{test_case} - {test_exec}\n"
        "Applied Forces, Measured Forces, and Steps on "
        f"A{i + 1} \n"
        f" {tangent_steps.index[0].isoformat(timespec='seconds')[:-6]} -"
        f" {tangent_steps.index[-1].isoformat(timespec='seconds')[:-6]}"
    )

    ax.set_title(f"{title}")

    ax.set_ylabel("Force (N)", fontsize=12)
    ax.set_xlabel("Step", fontsize=12)
    ax.grid(which="major", axis="both", linestyle="--")
    ax.set_xlim([-1200, 1200])

    ax = axes[1]
    ax.errorbar(
        np.array(step_mean)[ap_idx],
        rms_ap,
        fmt="ro",
        xerr=np.array(step_std)[ap_idx],
        yerr=np.array(ap_force_std)[ap_idx],
        label="Applied Force",
    )
    ax.errorbar(
        np.array(step_mean)[m_idx],
        rms_m,
        fmt="bo",
        xerr=np.array(step_std)[m_idx],
        yerr=np.array(m_force_std)[m_idx],
        label="Measured Force",
    )

    ax.axhline(y=0, linestyle=":", color="k")
    ax.grid(which="major", axis="both", linestyle="--")
    ax.set_ylabel("$\Delta$Force (N)", fontsize=12)
    ax.set_xlabel("Step", fontsize=12)
    ax.grid(which="major", axis="both", linestyle="--")
    ax.set_xlim([-1200, 1200])

    ax.set_ylim(
        [
            min(np.concatenate((rms_ap, rms_m))) * 1.5,
            max((np.concatenate((rms_ap, rms_m))) * 1.5),
        ]
    )
    fig.tight_layout()
    fig.savefig("./Step_vs_Forces_on_A" f"{i}.png")